In [ ]:
#Set directory
setwd("")

In [ ]:
library(ggplot2)
library(umap)
library(gridExtra)
library(apeglm)
library(igraph)
library(pheatmap)
library(colorRamps)
library(ggrepel)
library(dplyr)
library(reshape2)
library(matlab)
library(rdist)
library(RColorBrewer)
library(factoextra)
library(randomForest)
library(ranger)
library(ggbiplot)
library(plotly)
library(rfUtilities)
library(ggfortify)

In [ ]:
#Load directories containing single cell intensities of patient T cells
dirs<-list.dirs("./", recursive=FALSE, full.names=TRUE)
dirs


In [ ]:
#Identifying and load .csv files in each directory 
files<-unlist(lapply(dirs, function(x) list.files(x, full.names=TRUE, pattern=".csv")))

#Reading files
samples<-lapply(files, function(x) read.csv(x, stringsAsFactors = FALSE))
                
#Removing outliers
#Scaling daata
samples_scaled<-lapply(samples, function(x) as.data.frame(scale(x, center=TRUE)))

#Identifying outliers. Outliers are defined as standard deviation greater than 3 or less than -3
outliers<-lapply(samples_scaled, function(y) unique(unlist(lapply(1:ncol(samples_scaled[[1]]), function(x) which(y[,x]>3|y[,x]<(-3)))))) 

#Filtering data to remove outliers
samples_filtered<-lapply(1:length(outliers), function(x) samples[[x]][-outliers[[x]],])
                        
#Downsampling to 2000 cells per condition
set.seed(1234)
samples_sampled<-lapply(samples_filtered, function(x) x[sample(nrow(x), 800),])
                        
#Determining file names. The function extracts names from the .csv filenames
filenames<-sapply(files, function(x) paste(strsplit(x, "_")[[1]][3], strsplit(x, "_")[[1]][4], sep="_")) 
                  
#Annotating samples
samples_Ann<-lapply(1:length(filenames), function(x) data.frame(samples_sampled[[x]], condition=filenames[x], stringsAsFactors = FALSE))

#Renaming column names with marker names
for(i in 1:length(samples_Ann)){
   colnames(samples_Ann[[i]])<-c("FSC-A", "FSC-H", "SSC-A", "SSC-H","CCR7", "CD8", "CD4", "CD25" ,"tEGFR", "LD", "PD1", "CD137", "CD45RA", "CD95", "TIM3", "CD3", "condition")
    }

#Combining the various conditions into one dataframe
samples_combined<-lapply(samples_Ann, function(x) as.data.frame(x))%>% bind_rows()

In [ ]:
#Running Kmeans
#the function below runs kmeans with increasing k from from k=2 to k=330. To save time and computation, different intervals are used 
kmeans_analysis<-lapply(c(seq(from = 2, to = 18, by = 2), seq(from = 20, to = 270, by = 50), seq(from = 290, to = 330, by = 20)), function(x) kmeans(samples_combined[,c(5:8,11:15)], x, iter.max=1000, nstart=100))

#the function below extracts the cluster number assigned to each cell for the interative kmeans analysis above
cell.cluster<-lapply(1:length(kmeans_analysis), function(x) as.factor(kmeans_analysis[[x]]$cluster))

#the followng function extracts the cluster centers for the kmeans analysis above
cluster.centers<-lapply(1:length(kmeans_analysis), function(x) kmeans_analysis[[x]]$centers)


In [ ]:
#specifying T cell cytotoxicity values for a given sample and condition
cytox_values<-c(93.226, 97.110, 50.946, 41.565, 29.241, 9.754, 51.810, 47.790, 31.791, 31.244, 36.548, 50.116, 60.114, 79.285, 88.656,
         79.639,74.055, 33.220, 27.729, 36.016, 25.467, 16.680, 15.218, 9.197, 70.970, 90.839, 96.268, 92.424, 95.323, 66.482)

#Creating a dataframe with single cell intensities and the corresponding cytox value for the condition.
#The following first assigns each cell with the same cytox value of a given condition
cytox<-data.frame(condition=unique(samples_combined$condition), cytox=cytox_values)

In [ ]:
#This function specifies the cluster a specific cell belongs to from the iterative kmeans performed above
samples_cluster<-lapply(1:length(cell.cluster), function(x) data.frame(samples_combined, cluster=cell.cluster[[x]]))

#the following functions assign a cytox value for each cell of the iterative kmeans calculation
for(i in 1:length(samples_cluster)){
samples_cluster[[i]]$Cytox<-rowSums(sapply(1:nrow(cytox), function(x) 
ifelse(samples_cluster[[i]]$condition==cytox$condition[x], cytox$cytox[x],0)))
}
       

In [ ]:
#writing to file
clus<-c(seq(from = 2, to = 18, by = 2), seq(from = 20, to = 270, by = 50), seq(from = 290, to = 330, by = 20))

lapply(1:length(samples_cluster), function(x) write.csv(samples_cluster[[x]], 
paste0("./RandomForest_AC/Files_combined_Intensities/Samples_",paste0(clus[x], ".csv"))))


In [ ]:
#Extracting stimulation levels for each cell
for (i in 1:length(samples_cluster)){
samples_cluster[[i]]$Stimulation<-as.numeric(sapply(samples_cluster[[i]]$condition, function(y) paste0("0.", strsplit(y, split="_")[[1]][2])))*10
}

In [ ]:
#Calculating the mean attributes of markers and T cell cytotoxicity per cluster
cluster.feature<-lapply(samples_cluster, function(x) as.data.frame(x%>%group_by(cluster)%>%dplyr::summarize_if(is.numeric,mean, na.rm = TRUE)))

In [ ]:
#Running Random Forest regression on each iterative k-means, with independent variables being the mean marker intensities, and the output being the mean cytox per cluster 
rf_classifier <- lapply(cluster.feature, function(i) randomForest(x=i[,c(6:9, 12:16)], y=as.numeric(i[,"Cytox"]), data=i, ntree=1001, importance=TRUE))

#Extracting the pseudo R-squared (rsq) value, which is defined as 1 - mse / Var(y).
rf_rsq<-sapply(1:length(rf_classifier), function(x) rf_classifier[[x]]$rsq%>%dplyr::last())


In [ ]:
#The functions below generate an elbow plot to determine the optimal number of kmeans k clusters.
#The optimal number of kmeans clusters is the minimum above which the Random Forest rsq value increases minimally

#Extracting cluster numbers from file names and creating a dataframe with cluster number and the corresponding rsq value
cluster_number<-clus
rf_rsq.df<-data.frame(cluster_number=cluster_number, rsq=rf_rsq)

#Generating elbow plot
ggplot(rf_rsq.df, aes(x=cluster_number, y=rsq))+geom_point()+geom_line(color="red")+theme_classic()+labs(x="Number of Clusters", y="Variance Explained")+ylim(0,.8)

In [ ]:
#Optimal Numer of clusters=220
#Naming various dataframes contained within "cluster.feature" with their k cluster
names(cluster.feature)<-cluster_number

#Specifying the appropriate number of clusters based on the random forest elbow plot
x="220"

#Selecting dataframe which have the optimal k clusters 
cluster.feature_optimum<-cluster.feature[[x]]

In [ ]:
#Scaling columns to categorize into high, medium and low cytox
cluster.feature_optimum.scale<-as.data.frame(scale(cluster.feature_optimum[,c(6:9,12:16,18,19)]))
cluster.feature_optimum.scale$cluster<-paste("Clus", cluster.feature_optimum$cluster, sep="_")

#Group Clusters by Low, Medium, High Cytox
cluster.feature_optimum.scale$Level<-ifelse(cluster.feature_optimum.scale$Cytox<=(-1), "Low", ifelse(cluster.feature_optimum.scale$Cytox>=1, "High", "Moderate"))

#Finding characterstic marker expression for High, Moderate and Low cytox
marker_cytox<-as.data.frame(cluster.feature_optimum.scale%>%group_by(Level)%>%summarise_if(is.numeric, mean))
rownames(marker_cytox)<-marker_cytox$Level
marker_cytox$Level<-factor(marker_cytox$Level, levels=c("High", "Moderate", "Low"))

#Arranging rows according to cytox levels and plotting
marker_cytox<-marker_cytox[levels(marker_cytox$Level),]
marker_cytox<-data.frame(marker_cytox[, c(1:10)], marker_cytox[, c("Stimulation", "Cytox")])

pheatmap(marker_cytox[,c(2:11)], scale="none", fontsize_col = 11, fontsize_row = 10, color = colorRampPalette(rev(brewer.pal(n = 11, name ="RdYlBu")))(1000), cluster_cols=FALSE, cluster_rows=FALSE, border_color=NA, cellwidth=30, cellheight=30)


In [ ]:
#Plotting heatmaps without grouping according to High, Medium or Low cytox
cluster.feature_optimum.scale.order<-cluster.feature_optimum.scale%>%dplyr::arrange(desc(Cytox))
cluster.feature_optimum.scale.order<-data.frame(cluster.feature_optimum.scale.order[, c(1:9)],cluster.feature_optimum.scale.order[, c("Stimulation", "Cytox")])

pheatmap(cluster.feature_optimum.scale.order[,c(1:11)], scale="none", fontsize_col = 11, fontsize_row = 10, color = colorRampPalette(rev(brewer.pal(n = 11, name ="RdYlBu")))(1000), cluster_cols=FALSE, cluster_rows=FALSE, border_color=NA, cellwidth=30, show_rownames=FALSE)